kerasのpretrained modelsの精度検証

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('..')

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Activation, Flatten, Dense, BatchNormalization
from keras.layers.convolutional import Conv2D

from src.Utils import loadpkl, my_iou_metric, upsample, downsample, IMG_SIZE_TARGET, IMG_SIZE_ORI

img_size_target=101

In [7]:
train_df = loadpkl('../output/train_df.pkl')
train_df.head()

,z,images,masks,coverage,coverage_class
id,,,,,
575d24d81d,843,"[[0.525490196078, 0.513725490196, 0.5254901960...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.000000,0
a266a2a9df,794,"[[0.341176470588, 0.376470588235, 0.3333333333...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.504950,6
75efad62c1,468,"[[0.56862745098, 0.466666666667, 0.32549019607...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.993334,10
34e51dba6a,727,"[[0.541176470588, 0.474509803922, 0.3960784313...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.149201,2
4875705fb0,797,"[[0.0666666666667, 0.078431372549, 0.090196078...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.042839,1


In [62]:
X = np.array(train_df.images.map(upsample).tolist()).reshape(-1, IMG_SIZE_TARGET, IMG_SIZE_TARGET, 1)
Y = np.array(train_df.masks.tolist()).reshape(-1, IMG_SIZE_ORI, IMG_SIZE_ORI, 1)
cov = train_df.coverage.values
depth = train_df.z.values

# coverage_class以外のカラム名
feats = [f for f in train_df.columns if f not in ['coverage_class']]
ids_train, ids_valid, x_train, x_valid, y_train, y_valid,\
cov_train, cov_test, depth_train, depth_test = train_test_split(train_df.index.values,
                                                                X,
                                                                Y,
                                                                train_df.coverage.values,
                                                                train_df.z.values,
                                                                test_size=0.2, stratify=train_df.coverage_class, random_state= 1234)

x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)

x_train = np.repeat(x_train,3,axis=3)
x_valid = np.repeat(x_valid,3,axis=3)


In [63]:
model_resnet = ResNet50(include_top=False, weights='imagenet')
for layer in model_resnet.layers:
    layer.traiable=False

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [71]:
input_layer = model_resnet.input
x = model_resnet.output
x = BatchNormalization(axis=1)(x)
x = Conv2D(1, (1, 1), padding="same", activation=None, name="prediction")(x)
x= Activation('sigmoid')(x)

model = Model(input_layer, x)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[my_iou_metric])

ValueError: Axis 1 of input tensor should have a defined dimension but the layer received an input with shape (None, None, None, 2048).

In [67]:
history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid],
                    epochs=200,
                    batch_size=32,
#                    callbacks=[early_stopping, model_checkpoint, reduce_lr],
                    verbose=1)

Train on 6400 samples, validate on 800 samples
Epoch 1/200


InvalidArgumentError: Incompatible shapes: [32,4,4,1] vs. [32,101,101,1]
	 [[Node: loss_9/activation_155_loss/logistic_loss/mul = Mul[T=DT_FLOAT, _class=["loc:@train...ad/Reshape"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss_9/activation_155_loss/Log-0-0-TransposeNCHWToNHWC-LayoutOptimizer, _arg_activation_155_target_0_1/_12687)]]

In [56]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [59]:
model.output_shape

(None, None, None, 1)